导入库

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import time
import datetime
from sklearn.model_selection import train_test_split

import cv2

import h5py
%matplotlib inline

random_state = 424
dataset_path = 'dataset'

这个数据集是26位司机的各种样子的照片。所以我按照司机来划分训练集和验证集。

In [2]:
# 读取csv文件
dataset = pd.read_csv('dataset/driver_imgs_list.csv')

# 将前21为司机的照片作为训练集，后五位司机的照片作为测试集
drive_id_train = dataset['subject'].unique()[:-2]
drive_id_test = dataset['subject'].unique()[-2:]

# 组合路径
dataset['path']=dataset[['classname', 'img']].apply(lambda x: '/'.join(x), axis=1)

# 将路径转换成list格式
train_path = dataset['path'].loc[dataset['subject'].isin(drive_id_train)].tolist()
test_path = dataset['path'].loc[dataset['subject'].isin(drive_id_test)].tolist()

In [5]:
import os
import shutil


if os.path.exists('dataset/train2'):
    print('split dataset exist')
else:
    os.mkdir('dataset/train2')
    os.mkdir('dataset/valid2')
    for i in range(10):
        os.mkdir('dataset/train2/c'+str(i))
        os.mkdir('dataset/valid2/c'+str(i))
    # 复制图片到新的数据集
    if os.path.exists('dataset/train2'):
        print('train dataset copy begain')
        for filename in tqdm(train_path):
            shutil.copyfile('dataset/train/'+filename, 'dataset/train2/'+filename)
    if os.path.exists('dataset/valid2'):
        print('test dataset copy begain')
        for filename in tqdm(test_path):
            shutil.copyfile('dataset/train/'+filename, 'dataset/valid2/'+filename)




split dataset exist


读取数据(暂时不用)

In [23]:
def get_img(path, image_size):
    img = cv2.imread(path)
    # Reduce size
    resized = cv2.resize(img, image_size)
    return resized

def load_data(image_size, data_path):
    X_train = []
    y_train = []
    X_test = []
    y_test = []

    print('Read train images')
    for j in tqdm(range(10)):
        print('Load folder c{}'.format(j))
        path = os.path.join(data_path, 'train2',
                            'c' + str(j), '*.jpg')
        files = glob.glob(path)
        for fl in files:
            flbase = os.path.basename(fl)
            img = get_img(fl, image_size)
            X_train.append(img)
            y_train.append(j)
            
    print('Read test images')
    for j in tqdm(range(10)):
        print('Load folder c{}'.format(j))
        path = os.path.join(data_path, 'valid2',
                            'c' + str(j), '*.jpg')
        files = glob.glob(path)
        for fl in files:
            flbase = os.path.basename(fl)
            img = get_img(fl, image_size)
            X_test.append(img)
            y_test.append(j)
    # Normalize a list of sample image data in the range of 0 to 1
    X_train = np.array(X_train)
    X_train = X_train.astype('float32')
    # One hot encode a list of sample labels. Return a one-hot encoded vector for each label.
    y_train = np.eye(10)[y_train]
    
    X_test = np.array(X_test)
    X_test = X_train.astype('float32')
    # One hot encode a list of sample labels. Return a one-hot encoded vector for each label.
    y_test = np.eye(10)[y_test]
    
    
    return X_train, X_test, y_train, y_test


首先导入不包含全连接层的VGG16模型

In [8]:
from keras.models import Sequential, Model
from keras.applications import *
from keras.layers import Dropout, Flatten, Dense, Input, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

train_dir = 'dataset/train2'  # 训练集数据
val_dir = 'dataset/valid2' # 验证集数据
nb_classes = len(glob.glob(train_dir + "/*"))  # 分类数


batch_size = 64
epochs = 3
# def VGG16_model(X_train, y_train, X_test, y_test):
def run_a_model(BASE_MODEL, input_shape, fine_tune_layer):
    input_weight = input_shape[0]
    input_height = input_shape[1]
    input_tensor = Input((input_height, input_weight, 3))
    base_model = BASE_MODEL(input_tensor=Input((input_height, input_weight, 3)),
                            weights='imagenet', 
                            include_top=False, 
                            input_shape=(input_weight, input_height, 3)
                           )
    
    
    x = input_tensor
    '''
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x) #new FC layer, random init
    predictions = Dense(10, activation='softmax')(x) #new softmax layer
    model = Model(input=base_model.input, output=predictions)
    '''
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='softmax')(x)
    model = Model(base_model.input, x)    
    print("total layer count {}".format(len(base_model.layers)))
    
    # 冻结基础层
    '''
    for layer in base_model.layers:
        layer.trainable = False
    '''
    for i in range(fine_tune_layer):
        model.layers[i].trainable = False
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    train_gen = ImageDataGenerator(
        rotation_range=10.,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
    )
    
    test_gen = ImageDataGenerator()
    
    train_generator = train_gen.flow_from_directory(train_dir, (input_weight, input_height), 
                                                    shuffle=True, batch_size=batch_size, class_mode='categorical')
    test_generator = test_gen.flow_from_directory(val_dir, (input_weight, input_height), 
                                                  shuffle=True, batch_size=batch_size, class_mode='categorical')
    
    
    steps_train_sample = train_generator.samples // batch_size + 1
    steps_valid_sample = test_generator.samples // batch_size + 1

    model.fit_generator(
        train_generator,
        steps_per_epoch=steps_train_sample,
        epochs=epochs,
        validation_data=test_generator,
        validation_steps=steps_valid_sample)
    model.save("models/model_%s.h5"%BASE_MODEL.func_name)

    return

Using TensorFlow backend.


In [9]:
# VGG16_model(X_train, y_train, X_test, y_test)
run_a_model(VGG16, (224, 224), 18)

total layer count 19
Found 20787 images belonging to 10 classes.
Found 1637 images belonging to 10 classes.
Epoch 1/3
325/325 [==============================] - 350s - loss: 9.2516 - acc: 0.1748 - val_loss: 5.4419 - val_acc: 0.3213
Epoch 2/3
325/325 [==============================] - 337s - loss: 4.4143 - acc: 0.3438 - val_loss: 1.9947 - val_acc: 0.4740
Epoch 3/3
325/325 [==============================] - 336s - loss: 2.3864 - acc: 0.4365 - val_loss: 2.3632 - val_acc: 0.4184


In [13]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

def write_gap(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))
    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory("dataset/train2", image_size, shuffle=False, 
                                              batch_size=16)
    test_generator = gen.flow_from_directory("dataset/valid2", image_size, shuffle=False, 
                                             batch_size=16, class_mode=None)
    train = model.predict_generator(train_generator, 18587)
    test = model.predict_generator(test_generator, 3837)
    with h5py.File("gap_%s.h5"%MODEL.func_name) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)


In [14]:
write_gap(applications.VGG16, (224, 224))

Found 18587 images belonging to 10 classes.
Found 3837 images belonging to 10 classes.


KeyboardInterrupt: 

基础预测

In [30]:
from keras.models import load_model
model = load_model('models/model_VGG16.h5')

In [35]:
def load_test(img_rows, img_cols):
    print('Read test images')
    path = os.path.join(dataset_path, 'test', '*.jpg')
    files = glob.glob(path)
    X_test = []
    X_test_id = []
    total = 0
    thr = math.floor(len(files)/10)
    for fl in tqdm(files):
        flbase = os.path.basename(fl)
        img = get_img(fl, (img_rows, img_cols))
        X_test.append(img)
        X_test_id.append(flbase)
        total += 1

    return X_test, X_test_id


def make_predictions(MODEL, image_size, batch_size):
    gen = ImageDataGenerator()
    path_test_data = 'dataset/valid2/'
    test_generator = gen.flow_from_directory(path_test_data,  image_size, shuffle=False, 
                                             batch_size=batch_size, class_mode=None)
    y_predictions = MODEL.predict_generator(test_generator,  steps=test_generator.samples//batch_size+1,  verbose=1)
    y_predictions = y_predictions.clip(min=0.005, max=0.995)
    
    test_id = list()
    for i, file_name in enumerate(test_generator.filenames):
        if i >5:
            break
        print(file_name)
        # name = fname[fname.rfind('/')+1:]
        # l.append( [name, *y_pred[i]] )
        
    
    return y_predictions

def create_submission(predictions, test_id):
    result1 = pd.DataFrame(predictions, columns=['c0', 'c1', 'c2', 'c3',
                                                 'c4', 'c5', 'c6', 'c7',
                                                 'c8', 'c9'])
    result1.loc[:, 'img'] = pd.Series(test_id, index=result1.index)
    now = datetime.datetime.now()
    if not os.path.isdir('subm'):
        os.mkdir('subm')
    suffix = str(now.strftime("%Y-%m-%d-%H-%M"))
    sub_file = os.path.join('subm', 'submission_' + suffix + '.csv')
    result1.to_csv(sub_file, index=False)

In [36]:
make_predictions(model, (224, 224), 64)

Found 1637 images belonging to 10 classes.
26/26 [==============================] - 24s    
c0/img_99098.jpg
c0/img_77590.jpg
c0/img_40733.jpg
c0/img_50553.jpg
c0/img_21699.jpg
c0/img_35189.jpg


array([[ 0.005     ,  0.005     ,  0.00754303, ...,  0.11889337,
         0.005     ,  0.82859623],
       [ 0.01155962,  0.005     ,  0.03942531, ...,  0.005     ,
         0.00666136,  0.9344064 ],
       [ 0.00761088,  0.005     ,  0.01573017, ...,  0.005     ,
         0.01649698,  0.08067871],
       ..., 
       [ 0.005     ,  0.005     ,  0.005     , ...,  0.31859452,
         0.03302443,  0.64018369],
       [ 0.005     ,  0.02194109,  0.00781832, ...,  0.005     ,
         0.00800699,  0.80524421],
       [ 0.005     ,  0.01575794,  0.01929104, ...,  0.005     ,
         0.01956864,  0.61595052]], dtype=float32)

In [26]:
# X_test_to_pred_test , X_test_id_test = load_test(224, 224)
X_test_to_pred = np.array(X_test_to_pred)
print("pred begin")
start = time.time()
y_pred = model.predict(X_test_to_pred, verbose=1)
end = time.time()
print(end - start)

pred begin
18880/79726 [======>.......................] - ETA: 1050s

KeyboardInterrupt: 

In [114]:
create_submission(y_pred, X_test_id)